In [1]:
import sys


no_of_cpu =4
max_cores =8
executor_mem = '26g'


Job_Name = 'Nandan  :customer_web_data'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
17999,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpcustcoe"
pwd_prod="gpcustcoe@123"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
         "dbschema": "registers","dbtable": "premium_register",
         "partitionColumn":"row_num","partitions":16,
         "server.port":"1150-1170"}

premium_reg = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.selectExpr("channel","producer_cd","policy_no","certificate_no","pol_incept_date" , 'VEHICLE_REGISTRATION_NO',
            "pol_exp_date" , "customer_id", "product_name", "product_cd", 'num_reference_no as reference_num', 'timestamp')\
.filter((col('customer_id').isNotNull()))\
.filter(col('timestamp')>='2022-01-01')\
.filter(col('timestamp')<='2023-12-31')\
.filter(col('pol_exp_date')>current_date())\
.filter(col('product_cd').isin('2154', '3124', '3184', '3121','3186', '3122', '1603','1601','1602', '1604','1606','288901','2151',
                                '20807','2888','288802','3188','3189','140703','4260','2804','4277','4251','288805','288804','2896'))

premium_reg=premium_reg.repartition('customer_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "mc_tbl_bulk_detail_prod_corp",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

mbdp_corp = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("source_customer_id", "dc_unified_id")

mbdp_corp.columns

In [ ]:
ci_premium_reg=premium_reg.join(mbdp_corp, premium_reg.customer_id==mbdp_corp.source_customer_id)

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "mc_tbl_master_prod_corp",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

mmp_corp = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .filter(col("dc_active_flag").isin("1"))\
                   .selectExpr( "psnt_address_state", "dc_unified_id", 'dc_active_flag', 'psnt_address_city', 'psnt_address_zip','gstin',
                              'full_name','first_name', 'middle_name', 'last_name', 'pan', 'passport', 'email_primary', 
                               'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3', 'email_secondary', 'mobile_primary', 'mobile_secondary' )

mmp_corp.columns

In [ ]:
ci_premium_reg=ci_premium_reg.join(mmp_corp, ci_premium_reg.dc_unified_id==mmp_corp.dc_unified_id, 'inner').drop(mmp_corp.dc_unified_id).drop(ci_premium_reg.source_customer_id)

In [ ]:
ci_premium_reg.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "mastercraft_bulk_detail_prod",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

mbdp = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("source_customer_id", "dc_unified_id")

mbdp.columns

In [ ]:
ci_premium_reg_ret=premium_reg.join(mbdp, premium_reg.customer_id==mbdp.source_customer_id).drop(mbdp.source_customer_id)

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "mastercraft_master_prod",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

mmp = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .filter(col("dc_active_flag").isin("1"))\
                   .selectExpr( "psnt_address_state", "dc_unified_id", 'birth_date', 'dc_active_flag', 'psnt_address_city', 'psnt_address_zip','gstin',
                              'full_name','first_name', 'middle_name', 'last_name','cleansed_pan as pan', 'cleansed_passport as passport', 'email_primary', 
                               'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3', 'email_secondary', 'gender', 'mobile_primary', 'mobile_secondary' )

mmp.columns

In [ ]:
ci_premium_reg_ret=ci_premium_reg_ret.join(mmp, ci_premium_reg_ret.dc_unified_id==mmp.dc_unified_id, 'inner').drop(mmp.dc_unified_id)

In [ ]:
ci_premium_reg_kk=ci_premium_reg_ret.select('channel', 'producer_cd', 'policy_no', 'certificate_no', 'pol_incept_date', 'VEHICLE_REGISTRATION_NO', 'pol_exp_date', 'customer_id', 'product_name', 'product_cd', 'reference_num', 'timestamp','psnt_address_state', 'dc_unified_id', 'dc_active_flag', 'psnt_address_city', 'psnt_address_zip', 'gstin', 'full_name', 'first_name', 'middle_name', 'last_name', col('pan'), col('passport'), 'email_primary', 'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3', 'email_secondary', 'mobile_primary', 'mobile_secondary', 'customer_id')


In [ ]:
ci_master=ci_premium_reg_kk.union(ci_premium_reg.select('channel', 'producer_cd', 'policy_no', 'certificate_no', 'pol_incept_date', 'VEHICLE_REGISTRATION_NO', 'pol_exp_date', 'customer_id', 'product_name', 'product_cd', 'reference_num', 'timestamp','psnt_address_state', 'dc_unified_id', 'dc_active_flag', 'psnt_address_city', 'psnt_address_zip', 'gstin', 'full_name', 'first_name', 'middle_name', 'last_name',  col('pan'), col('passport'), 'email_primary', 'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3', 'email_secondary', 'mobile_primary', 'mobile_secondary', 'customer_id'))
ci_master.columns

In [ ]:
ci_master=ci_master.join(ci_premium_reg_ret.select('dc_unified_id','birth_date','gender'), ci_master.dc_unified_id==ci_premium_reg_ret.dc_unified_id, 'inner').drop(ci_premium_reg_ret.dc_unified_id)
ci_master.columns

In [ ]:
ci_master=ci_master.withColumn('email', coalesce('email_primary', 'email_secondary'))\
.withColumn('mobile', coalesce('mobile_primary', 'mobile_secondary'))

In [ ]:
#ci_master.persist()

In [ ]:
ci =ci_master

In [ ]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123"
    ,"dbschema":"customermart"
    ,"dbtable": "customer_demographics",
         "server.port":"1150-1170"}


cust_demog_retail_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.selectExpr('golden_id','ckyc','kyc_id','kyc_verfication_flag','kyc_datetime_stamp').filter((col('golden_id').isNotNull()))\
.drop_duplicates()


gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
    "dbschema":"customermart"
    ,"dbtable": "corp_customer_demographics",
         "server.port":"1150-1170"}

demog_corp_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('golden_id','ckyc','kyc_id','kyc_verfication_flag','kyc_datetime_stamp')\
.drop_duplicates().filter((col('golden_id').isNotNull()))


kyc=cust_demog_retail_df.union(demog_corp_df)

In [ ]:
ci= ci.join(kyc, ci.dc_unified_id == kyc.golden_id, "left_outer")\
.drop(kyc.golden_id)

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "policy_dh_risk_headers_mot",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

mot_uw_mot = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("reference_num", "nominee_name as nominee_name_mot", 'engine_number', 'chassis_number', 'nomineeage as nomineeage_mot', 'relationship as relationship_mot', 'relationship_to_the_insured as relationship_to_the_insured_mot')\
                   .dropDuplicates()

mot_uw_mot.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "policy_dh_risk_details_hlt",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

hlt = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("reference_num", "insured_age as insured_age_hlt", 'insured_dob as insured_dob_hlt', 'insured_name as insured_name_hlt', 'insured_rel_with_proposer as insured_relationship_hlt')

hlt.columns

In [ ]:
hlt=hlt.withColumn('insured_name_hlt', trim(hlt.insured_name_hlt))
hlt = hlt.dropDuplicates(["reference_num",'insured_name_hlt'])
hlt.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_hlt",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

uw_hlt = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("num_reference_number", "nomineename as nomineename_hlt",'age as nom_age_hlt', 'dateofbirth as nom_dob_hlt', "relationshipwithinsured as nomi_rel_hlt")\
                   .filter(col('nomineename_hlt').isNotNull())

uw_hlt.columns

In [ ]:
uw_hlt=uw_hlt.withColumn('nomineename_hlt', trim(uw_hlt.nomineename_hlt))
uw_hlt = uw_hlt.dropDuplicates(["num_reference_number",'nomineename_hlt'])
uw_hlt.columns

In [ ]:
hlt_uw_hlt=hlt.join(uw_hlt, hlt.reference_num==uw_hlt.num_reference_number, "left_outer").drop(uw_hlt.num_reference_number)

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "policy_dh_risk_details_trv",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

trv = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("reference_num", 'insured_name as insured_name_trv')

trv.columns

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "policy_dh_risk_details_lta",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

lta = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("reference_num", 'nominee_name as nominee_name_lta', 'relationship as relationship_lta', 'insured_first_name as insured_first_name_lta', 'insured_middle_name as insured_middle_name_lta', 'insured_last_name as insured_last_name_lta', 'dob as dob_lta')\
                   .dropDuplicates()

lta.columns

In [ ]:
#lta.filter(col('reference_num').isin('202310100076170')).show()

In [ ]:
lta=lta.withColumn('insured_name_lta', concat_ws(' ', lta.insured_first_name_lta, lta.insured_middle_name_lta, lta.insured_last_name_lta))
lta.columns

In [ ]:
#lta=lta.withcolumn('nom_dob_lta', when(col('nominee_name_lta').isNotNull(), lit('dob_lta')).otherwise(lit(None)))

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "underwriting_dh_cnfgtr_otherdt_grid_tab_trv",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

uw_trv = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("num_reference_number", "nomineedob as nomineedob_trv", 'nomineename as nomineename_trv', 'nomineerelationship as nomineerelationship_trv', 'nomineegender as nomineegender_trv')

uw_trv.columns

In [ ]:
trv_uw_trv=trv.join(uw_trv, trv.reference_num==uw_trv.num_reference_number, "left_outer").drop(uw_trv.num_reference_number)

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "public",
                    "dbtable": "policy_gc_financier_details_tab",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

financier = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .selectExpr("num_reference_number as reference_num", "financier_name", 'financier_code')

financier.columns

In [ ]:
financier=financier.withColumn('isFinanced', when(col('financier_name').isNotNull(), lit(True)).otherwise(lit(False)))

In [ ]:
financier.columns

In [ ]:
ci=ci.join(lta, "reference_num","left_outer")
ci =ci.join(mot_uw_mot, "reference_num","left_outer")
ci =ci.join(hlt_uw_hlt, "reference_num","left_outer")
ci =ci.join(trv_uw_trv, "reference_num","left_outer")
ci =ci.join(financier, "reference_num","left_outer")

#ci.persist()

In [ ]:
#ci.filter(col('dob_lta').isNotNull()).show()

In [ ]:
ci = ci.dropDuplicates()
ci=ci.withColumn('lob', when(col('product_cd').isin('288901','2888','140703','4260','2804','4277','4251','288805','288802','288804','2896'), lit('Health'))\
                        .when(col('product_cd').isin('1604','1601','1606','1602','1603'), lit('Travel'))\
                        .when(col('product_cd').isin('2151'), lit('Marine'))\
                        .when(col('product_cd').isin('3122'), lit('2W'))\
                        .when(col('product_cd').isin('3184','3121','3124','3188','3189'), lit('4W')))

In [ ]:
ci=ci.withColumn('passportFileNumber', lit('NA').cast('string'))

In [ ]:
ci=ci.withColumn('nominee_name', coalesce(col('nomineename_hlt'), col('nominee_name_mot'), col('nomineename_trv'), col('nominee_name_lta')))\
.withColumn('nominee_dob', coalesce(col('nomineedob_trv'),col('nom_dob_lta'), col('nom_dob_hlt')))\
.withColumn('nominee_age', coalesce(col('nomineeage_mot'), col('nom_age_hlt')))\
.withColumn('nominee_gender', coalesce(col('nomineegender_trv')))\
.withColumn('nominee_relation', coalesce(col('nomi_rel_hlt'), col('relationship_mot'),col('relationship_to_the_insured_mot'),col('relationship_lta'), col('nomineerelationship_trv')))\
.withColumn('insured_name_travel', coalesce(col('insured_name_lta'), col('insured_name_trv')))

In [ ]:
# ci=ci.withColumn('pol_incept_date',date_format(to_utc_timestamp(col('pol_incept_date'),"UTC"),'yyyy-MM-dd'))\
# .withColumn('pol_exp_date',date_format(to_utc_timestamp(col('pol_exp_date'),"UTC"),'yyyy-MM-dd'))\
# .withColumn('dob_lta',from_unixtime(unix_timestamp(col('timestamp')),'yyyy-MM-dd'))
# .withColumn('insured_dob_hlt',date_format(to_utc_timestamp(col('insured_dob_hlt'),"UTC"),'yyyy-MM-dd'))\
# .withColumn('nominee_dob',date_format(to_utc_timestamp(col('nominee_dob'),"UTC"),'yyyy-MM-dd'))\
# .withColumn('birth_date',from_unixtime(unix_timestamp(col('timestamp')),'yyyy-MM-dd'))
# .withColumn('kyc_datetime_stamp',from_unixtime(unix_timestamp(col('kyc_datetime_stamp')),'yyyy-MM-dd HH:mm:ss'))\
# .withColumn('timestamp',from_unixtime(unix_timestamp(col('timestamp')),'yyyy-MM-dd HH:mm:ss'))

# #.withColumn('dob_lta',date_format(to_utc_timestamp(col('dob_lta'),"UTC"),'yyyy-MM-dd'))\


In [ ]:
#ci.filter(col('dob_lta').isNotNull()).show()

In [ ]:
ci=ci.filter(col('lob').isNotNull())\
.filter(col('mobile').isNotNull())

In [ ]:
df_distinct=ci.withColumn('primary_key', concat_ws('',trim(col('dc_unified_id')),trim(col('customer_id')), trim(col('policy_no')),
                                                       trim(col('certificate_no')),trim(col('insured_name_hlt')),trim(col('insured_name_travel')))).dropDuplicates()

In [ ]:
from pyspark.sql.window import Window

df_distinct = df_distinct.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("timestamp").desc())))

df_distinct = df_distinct.filter(F.col("rn") == 1).drop("rn").drop("primary_key")

In [ ]:
df_distinct=df_distinct.select('lob',col('dc_unified_id').alias('golden_id'),col('customer_id').alias('source_customer_id'),
             'reference_num', 'policy_no', 'certificate_no','channel', 'producer_cd', 'product_name', 'product_cd', 
             'full_name', 'first_name', 'middle_name', 'last_name','pol_incept_date','pol_exp_date', 'timestamp',  
             'psnt_address_state', 'psnt_address_city', 'psnt_address_zip','psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3',
             'gstin', 'pan','passport','passportFileNumber','kyc_id', 'kyc_verfication_flag', 'ckyc', 'kyc_datetime_stamp',
             'birth_date','gender', 'email', 'mobile', 
             'engine_number','chassis_number', 'VEHICLE_REGISTRATION_NO','insured_name_travel','dob_lta',
             'insured_name_hlt','insured_age_hlt', 'insured_dob_hlt','insured_relationship_hlt',
             'financier_name', 'financier_code','isFinanced',
             'nominee_name', 'nominee_dob', 'nominee_age', 'nominee_gender', 'nominee_relation')

In [ ]:
#df_distinct=df_distinct.filter(col('lob').isNotNull())

In [ ]:
df_distinct.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
#records=df_distinct.count()
#records

In [ ]:
#df_distinct.filter(col("product_cd").isin('2151')).count()
#df_distinct.select('product_cd').distinct().show(100, False)

In [ ]:
#df_distinct.filter(col('birth_date').isNotNull()).show()

In [6]:
output_index = 'website_api_staging_1601' #website_api_staging_2223 # website_api_staging_051223
schema = 'customermart'

import sys
try:
    
     df_distinct.write.option("truncate", "true").format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 'success'
    print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df_distinct' is not defined

In [18]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.website_api_staging_1601",
         "server.port":"1150-1170"}


df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['lob', 'golden_id', 'source_customer_id', 'reference_num', 'policy_no', 'certificate_no', 'channel', 'producer_cd', 'product_name', 'product_cd', 'full_name', 'first_name', 'middle_name', 'last_name', 'pol_incept_date', 'pol_exp_date', 'timestamp', 'psnt_address_state', 'psnt_address_city', 'psnt_address_zip', 'psnt_address_line_1', 'psnt_address_line_2', 'psnt_address_line_3', 'gstin', 'pan', 'passport', 'passportFileNumber', 'kyc_id', 'kyc_verfication_flag', 'ckyc', 'kyc_datetime_stamp', 'birth_date', 'gender', 'email', 'mobile', 'engine_number', 'chassis_number', 'VEHICLE_REGISTRATION_NO', 'insured_name_travel', 'dob_lta', 'insured_name_hlt', 'insured_age_hlt', 'insured_dob_hlt', 'insured_relationship_hlt', 'financier_name', 'financier_code', 'isFinanced', 'nominee_name', 'nominee_dob', 'nominee_age', 'nominee_gender', 'nominee_relation']

In [20]:
df.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[lob: string, golden_id: bigint, source_customer_id: string, reference_num: bigint, policy_no: string, certificate_no: string, channel: string, producer_cd: string, product_name: string, product_cd: string, full_name: string, first_name: string, middle_name: string, last_name: string, pol_incept_date: date, pol_exp_date: date, timestamp: date, psnt_address_state: string, psnt_address_city: string, psnt_address_zip: string, psnt_address_line_1: string, psnt_address_line_2: string, psnt_address_line_3: string, gstin: string, pan: string, passport: string, passportFileNumber: string, kyc_id: string, kyc_verfication_flag: string, ckyc: string, kyc_datetime_stamp: timestamp, birth_date: string, gender: string, email: string, mobile: string, engine_number: string, chassis_number: string, VEHICLE_REGISTRATION_NO: string, insured_name_travel: string, dob_lta: string, insured_name_hlt: string, insured_age_hlt: string, insured_dob_hlt: string, insured_relationship_hlt: string, financier

In [60]:
get_website_api_data=df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
get_website_api_data=get_website_api_data.withColumn('pol_incept_date',date_format(to_utc_timestamp(col('pol_incept_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('pol_exp_date',date_format(to_utc_timestamp(col('pol_exp_date'),"UTC"),'yyyy-MM-dd'))\
.withColumn('kyc_datetime_stamp',from_unixtime(unix_timestamp(col('kyc_datetime_stamp')),'yyyy-MM-dd HH:mm:ss'))\
.withColumn('timestamp',from_unixtime(unix_timestamp(col('timestamp')),'yyyy-MM-dd HH:mm:ss'))\
.withColumn('nominee_dob',to_date(col('nominee_dob'),'dd/MM/yyyy'))\
.withColumn('dob_lta',to_date(col('dob_lta'),'dd/MM/yyyy'))\
.withColumn('birth_date',to_date(col('birth_date'),'yyyy/MM/dd'))\
.withColumn('insured_dob_hlt',to_date(col('insured_dob_hlt'),'dd/MM/yyyy'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
get_website_api_data = get_website_api_data.withColumn("nominee_dob", date_format(unix_timestamp('nominee_dob' , "dd/MM/yyyy").cast("timestamp"),"yyyy-MM-dd"))\
.withColumn("dob_lta", date_format(unix_timestamp('dob_lta' , "dd/MM/yyyy").cast("timestamp"),"yyyy-MM-dd"))\
.withColumn("birth_date", date_format(unix_timestamp('birth_date' , "dd/MM/yyyy").cast("timestamp"),"yyyy-MM-dd"))\
.withColumn("insured_dob_hlt", date_format(unix_timestamp('insured_dob_hlt' , "dd/MM/yyyy").cast("timestamp"),"yyyy-MM-dd"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
get_website_api_data=get_website_api_data.withColumn('nominee_age', 
    when(col('nominee_age').isNull(), 
         floor(datediff(current_date(),col('nominee_dob'))/365.25))\
  .otherwise(col('nominee_age')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
get_website_api_data=get_website_api_data.withColumn("dob_lta",col('dob_lta').cast('string'))\
.withColumn("birth_date", col('birth_date').cast('string'))\
.withColumn("insured_dob_hlt", col('insured_dob_hlt').cast('string'))\
.withColumn("nominee_dob", col('nominee_dob').cast('string'))   

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
#get_website_api_data=get_website_api_data.withColumn('customer_age', )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#get_website_api_data.filter(col('nominee_age').isNotNull()).show()

In [ ]:
# get_website_api_data.filter(col('mobile').isin('8289936720','9611961139',
# '8289936720',
# '9811172414',
# '9432933141',
# '9072088018',
# '9910915599',
# '8971857977')).show()

In [ ]:
# get_website_api_data=get_website_api_data.withColumn('lob', when(col('product_cd').isin('288901','140703','4260','2804','4277','4251','288805','288802','288804','2896'), lit('Health'))\
#                         .when(col('product_cd').isin('1604','1601','1606','1602','1603'), lit('Travel'))\
#                         .when(col('product_cd').isin('2151'), lit('Marine'))\
#                         .when(col('product_cd').isin('3122'), lit('2W'))\
#                         .when(col('product_cd').isin('3184','3121','3124', '3188','3189'), lit('4W'))

In [ ]:
#get_website_api_data=get_website_api_data.withColumn('passportFileNumber', lit('NA').cast('string'))

In [ ]:
#get_website_api_data=get_website_api_data.select()

In [66]:
get_website_api_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- lob: string (nullable = true)
 |-- golden_id: long (nullable = true)
 |-- source_customer_id: string (nullable = true)
 |-- reference_num: long (nullable = true)
 |-- policy_no: string (nullable = true)
 |-- certificate_no: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- producer_cd: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_cd: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- pol_incept_date: string (nullable = true)
 |-- pol_exp_date: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- psnt_address_state: string (nullable = true)
 |-- psnt_address_city: string (nullable = true)
 |-- psnt_address_zip: string (nullable = true)
 |-- psnt_address_line_1: string (nullable = true)
 |-- psnt_address_line_2: string (nullable = true)
 |-- psnt_address_line_3:

In [67]:
target_index='get_website_api_data'
print(target_index)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

get_website_api_data

In [68]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later

#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_website_api_data'
#########delete index if already exists########################################################################################################################################################################

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
target_doc_type

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'get_website_api_data'

In [70]:
# #########delete index ###############################################
# headers = {'Content-Type': 'application/json'}
# query = {"query":{"match_all":{}}}
# URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
# r = requests.delete(URL, data=json.dumps(query), headers=headers) 
# print(r)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [200]>

In [71]:
#########create index ##################################################################################################################################################################
headers = {'Content-Type': 'application/json'}
query_create= {"settings": { "index.number_of_shards": "1" }}
URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
r = requests.put(URL, data=json.dumps(query_create), headers=headers)
print(r)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<Response [200]>

In [ ]:
# mapping_output = {target_doc_type:{"properties": {
# "golden_id": {
# "type": "keyword"
# }, 
# "customer_id": {
# "type": "keyword"
# },
# "insured_name": {
# "type": "keyword"
# },
# "first_name": {
# "type": "keyword"
# },
# "middle_name": {
# "type": "keyword"
# },
# "last_name": {
# "type": "keyword"
# },
# "ckyc": {
# "type": "keyword"
# },
# "kyc_id": {
# "type": "keyword"
# },
# "kyc_verfication_flag": {
# "type": "keyword"
# },
# "kyc_datetime_stamp": {
# "type": "keyword",
# #     "format": "yyyy-MM-dd HH:mm:ss"
# },
# "birth_date": {
# "type": "keyword",
# #     "format": "yyyy-MM-dd HH:mm:ss"
# },
# "pan": {
# "type": "keyword"
# },
# "passport": {
# "type": "keyword"
# },
# "email": {
# "type": "keyword"
# },
# "mobile": {
# "type": "keyword"
# },
# "address_line_1": {
# "type": "keyword"
# },
# "address_line_2": {
#     "type": "keyword"
# },
# "address_line_3": {
# "type": "keyword"
# },
# "city": {
# "type": "keyword"
# },
# "state": {
# "type": "keyword"
# },
# "country": {
# "type": "keyword"
# },
# "pin_code": {
# "type": "keyword"
# },
# "customer_type": {
# "type": "keyword"
# },
# "msme_no": {
# "type": "keyword"
# },
# "gstin": {
# "type": "keyword"
# },
# "district": {
# "type": "keyword"
# }
#     }
# }
# }


# URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index+'/_mapping/'+target_doc_type
# headers = {'Content-Type': 'application/json'}
# r = requests.put(URL, data=json.dumps(mapping_output), headers=headers)
# print(r)



In [72]:
target_index_doc_type = target_index +'/'+ target_doc_type

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
target_index_doc_type

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'get_website_api_data/get_website_api_data'

In [74]:
df_main=get_website_api_data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df_main=df_main.drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df_main.persist()
#df_main.count()

In [76]:
try:
     df_main.write.format('org.elasticsearch.spark.sql').mode('overwrite').option('es.index.auto.create', 'true').option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type).save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [ ]:
#.withColumn('birth_date',date_format(to_utc_timestamp(col('birth_date'),"UTC"),'yyyy-MM-dd'))\
#.withColumn('kyc_datetime_stamp',from_unixtime(unix_timestamp(col('kyc_datetime_stamp')),'yyyy-MM-dd HH:mm:ss'))\

In [77]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…